In [6]:
"""
The program below is a tutorial program used to learn how to connect python with SQL.
It creates a database for an International Language School which provides professional
language lessons to corprorate clients.
It is based on work done by:
    Craig Dickson< https://www.freecodecamp.org/news/connect-python-with-sql/ >
"""

'\nThe program below is a tutorial program used to learn how to connect python with SQL.\nIt is based on work done by:\n    Craig Dickson< https://www.freecodecamp.org/news/connect-python-with-sql/ >\n'

In [1]:
# 1. Connecting to MySQL Server
import mysql.connector as mysql# Python library used to connect the script to our MySQL server
from mysql.connector import Error # Provides easy access to the error function
import pandas as pd # Python library used for data analysis
import getpass # to read password
from getpass import getpass
def create_server_connection(host_name, user_name, user_password):
    """
    This function lets us establish a connection to our server
    """
    connection = None # Closes any existing connections so that the sever doesn't become confused with multiple open connections
    try:
        connection = mysql.connect(
            host = host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection
pw = getpass("Password: ")
connection = create_server_connection("localhost","root", pw)

Password: ········
MySQL Database connection successful


In [4]:
# 2. Creating a New Database
def create_database(connection, query):
    """
    This function takes two arguments, a connection and a query, 
    executing the query in the server via the connection.
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query) # Executes our queries on the MySQL Server
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'") 
# Initialize query to create our database
create_database_query = """
DROP DATABASE IF EXISTS school;
CREATE DATABASE IF NOT EXISTS school;
"""
# Create our database
create_database(connection, create_database_query)

Database created successfully


In [2]:
# 3. Connecting to the Databse
def create_db_connection(host_name, user_name, user_password, db_name):
    """
    This function is used to connect to a specific, already created database in
    our MySQL Server
    """
    connecton = None
    try:
        connection = mysql.connect(
            host = host_name,
            user = user_name,
            passwd = user_password,
            database = db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection
db = "school"
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [19]:
# 4. Creating  Query Execution Function
def execute_query(connection, query):
    """
    The function takes our SQL queries , stored in Python as strings, and 
    passes them to the cursor.execute() method to execute them on the server.
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit() # Ensures that the commands detailed in our SQL queries are implemented
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [8]:
# 5. Creating Tables
# In triple quoted strings, unescaped newlines & quotes are allowed
create_teacher_table = """
CREATE TABLE teacher (
    teacher_id INT PRIMARY KEY,
    first_name VARCHAR(40) NOT NULL,
    last_name VARCHAR(40) NOT NULL,
    language_1 VARCHAR(3) NOT NULL,
    language_2 VARCHAR(3),
    dob DATE,
    tax_id INT UNIQUE,
    phone_no VARCHAR(20)
    );
"""

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Databse
execute_query(connection, create_teacher_table) # Execute our defined query

MySQL Database connection successful
Query successful


In [9]:
# 5b. Create the remaining tables
create_client_table = """
CREATE TABLE client(
    client_id INT PRIMARY KEY,
    client_name VARCHAR(40) NOT NULL,
    address VARCHAR(60) NOT NULL,
    industry VARCHAR(20)
);
"""
create_participant_table = """
CREATE TABLE participant(
    participant_id INT PRIMARY KEY,
    first_name VARCHAR(40) NOT NULL,
    last_name VARCHAR(40) NOT NULL,
    phone_no VARCHAR(20),
    client INT
);
"""
create_course_table = """
CREATE TABLE course(
    course_id INT PRIMARY KEY,
    course_name VARCHAR(40) NOT NULL,
    language VARCHAR(3) NOT NULL,
    level VARCHAR(2),
    course_length_weeks INT,
    start_date DATE,
    in_school BOOLEAN,
    teacher INT,
    client INT
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_client_table)
execute_query(connection, create_participant_table)
execute_query(connection, create_course_table)

MySQL Database connection successful
Query successful
Query successful
Query successful


In [10]:
# 5c. Define the relationships between the tables
# teacher : course -> 1:N // Add foreign key to course table(primary key of teacher table)
# client : course -> 1:N // Add foreign key to course table(primary key of client table)
# client : participant -> 1:N // Add foreign key to participant table(primary key of client table)
# participant : course -> N:M // Create new table takes_course and add two foreign keys to it(primary key of participant and client tables)
alter_participant = """
ALTER TABLE participant
ADD FOREIGN KEY(client)
REFERENCES client(client_id)
ON DELETE SET NULL;
"""

alter_course = """
ALTER TABLE course
ADD FOREIGN KEY(teacher)
REFERENCES teacher(teacher_id)
ON DELETE SET NULL;
"""

alter_course_again = """
ALTER TABLE course
ADD FOREIGN KEY(client)
REFERENCES client(client_id)
ON DELETE SET NULL;
"""

create_takescourse_table = """
CREATE TABLE takes_course (
  participant_id INT,
  course_id INT,
  PRIMARY KEY(participant_id, course_id),
  FOREIGN KEY(participant_id) REFERENCES participant(participant_id) ON DELETE CASCADE,
  FOREIGN KEY(course_id) REFERENCES course(course_id) ON DELETE CASCADE
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, alter_participant)
execute_query(connection, alter_course)
execute_query(connection, alter_course_again)
execute_query(connection, create_takescourse_table)

MySQL Database connection successful
Query successful
Query successful
Query successful
Query successful


In [11]:
# 6. Populating the Tables
pop_teacher = """
INSERT INTO teacher VALUES
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie',  'Martin',  'FRA', NULL,  '1970-02-17', 23456, '+491234567890'), 
(3, 'Steve', 'Wang',  'MAN', 'ENG', '1990-11-12', 34567, '+447840921333'),
(4, 'Friederike',  'Müller-Rossi', 'DEU', 'ITA', '1987-07-07',  45678, '+492345678901'),
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', '1963-05-30',  56789, '+491772635467'),
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', '1995-09-08',  67890, '+491231231232');
"""

pop_client = """
INSERT INTO client VALUES
(101, 'Big Business Federation', '123 Falschungstraße, 10999 Berlin', 'NGO'),
(102, 'eCommerce GmbH', '27 Ersatz Allee, 10317 Berlin', 'Retail'),
(103, 'AutoMaker AG',  '20 Künstlichstraße, 10023 Berlin', 'Auto'),
(104, 'Banko Bank',  '12 Betrugstraße, 12345 Berlin', 'Banking'),
(105, 'WeMoveIt GmbH', '138 Arglistweg, 10065 Berlin', 'Logistics');
"""

pop_participant = """
INSERT INTO participant VALUES
(101, 'Marina', 'Berg','491635558182', 101),
(102, 'Andrea', 'Duerr', '49159555740', 101),
(103, 'Philipp', 'Probst',  '49155555692', 102),
(104, 'René',  'Brandt',  '4916355546',  102),
(105, 'Susanne', 'Shuster', '49155555779', 102),
(106, 'Christian', 'Schreiner', '49162555375', 101),
(107, 'Harry', 'Kim', '49177555633', 101),
(108, 'Jan', 'Nowak', '49151555824', 101),
(109, 'Pablo', 'Garcia',  '49162555176', 101),
(110, 'Melanie', 'Dreschler', '49151555527', 103),
(111, 'Dieter', 'Durr',  '49178555311', 103),
(112, 'Max', 'Mustermann', '49152555195', 104),
(113, 'Maxine', 'Mustermann', '49177555355', 104),
(114, 'Heiko', 'Fleischer', '49155555581', 105);
"""

pop_course = """
INSERT INTO course VALUES
(12, 'English for Logistics', 'ENG', 'A1', 10, '2020-02-01', TRUE,  1, 105),
(13, 'Beginner English', 'ENG', 'A2', 40, '2019-11-12',  FALSE, 6, 101),
(14, 'Intermediate English', 'ENG', 'B2', 40, '2019-11-12', FALSE, 6, 101),
(15, 'Advanced English', 'ENG', 'C1', 40, '2019-11-12', FALSE, 6, 101),
(16, 'Mandarin für Autoindustrie', 'MAN', 'B1', 15, '2020-01-15', TRUE, 3, 103),
(17, 'Français intermédiaire', 'FRA', 'B1',  18, '2020-04-03', FALSE, 2, 101),
(18, 'Deutsch für Anfänger', 'DEU', 'A2', 8, '2020-02-14', TRUE, 4, 102),
(19, 'Intermediate English', 'ENG', 'B2', 10, '2020-03-29', FALSE, 1, 104),
(20, 'Fortgeschrittenes Russisch', 'RUS', 'C1',  4, '2020-04-08',  FALSE, 5, 103);
"""

pop_takescourse = """
INSERT INTO takes_course VALUES
(101, 15),
(101, 17),
(102, 17),
(103, 18),
(104, 18),
(105, 18),
(106, 13),
(107, 13),
(108, 13),
(109, 14),
(109, 15),
(110, 16),
(110, 20),
(111, 16),
(114, 12),
(112, 19),
(113, 19);
"""
connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, pop_teacher)
execute_query(connection, pop_client)
execute_query(connection, pop_participant)
execute_query(connection, pop_course)
execute_query(connection, pop_takescourse)

MySQL Database connection successful
Query successful
Query successful
Query successful
Query successful
Query successful


In [4]:
# 7. Reading Data
def read_query(connection, query):
    """
    The function pulls data from our database and feeds it into our
    python data pipeline as a tuple
    """
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [10]:
"""
Create a list grouping all courses that are taught at the client's office
and include the client's name and address
"""
query1 = """
SELECT course.course_id, course.course_name, course.language, client.client_name, client.address
FROM course
JOIN client
ON course.client = client.client_id
WHERE course.in_school = FALSE;
"""
# Create a connection to the database
connection = create_db_connection("localhost", "root", pw, db)
# Read the records
results = read_query(connection, query1)
# Declare a list variable to store the results
course_records = []
# Convert the results into a list of lists
for result in results:
    course_records.append(list(result))
# Print out the list
for item in course_records:
    print(item)

MySQL Database connection successful
[13, 'Beginner English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin']
[14, 'Intermediate English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin']
[15, 'Advanced English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin']
[17, 'Français intermédiaire', 'FRA', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin']
[19, 'Intermediate English', 'ENG', 'Banko Bank', '12 Betrugstraße, 12345 Berlin']
[20, 'Fortgeschrittenes Russisch', 'RUS', 'AutoMaker AG', '20 Künstlichstraße, 10023 Berlin']


In [14]:
# 8. Formatting Output into a pandas Dataframe
# Create a list to store our column headers
columns = ["Course Id", "Course Name", "Language", "Client Name", "Address"]
# Initialize a pandas dataframe
df = pd.DataFrame(course_records, columns = columns)
# Display our dataframe
display(df)

,Course Id,Course Name,Language,Client Name,Address
0,13,Beginner English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
1,14,Intermediate English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
2,15,Advanced English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
3,17,Français intermédiaire,FRA,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
4,19,Intermediate English,ENG,Banko Bank,"12 Betrugstraße, 12345 Berlin"
5,20,Fortgeschrittenes Russisch,RUS,AutoMaker AG,"20 Künstlichstraße, 10023 Berlin"


In [22]:
"""
Big Business Federation, one of our clients, is moving its premises to 
23 Fingiertweg, 14534 Berlin. Update the records
"""
update = """
UPDATE client
SET address = "23 Fingiertweg, 14534 Berlin"
WHERE client_id = 101;
"""
q1 = """
SELECT * 
FROM client;
"""
execute_query(connection, update)
connection = create_db_connection("localhost", "root", pw, db)
client = read_query(connection, q1)
client_records = []
for record in client:
    client_records.append(list(record))
h1 = ["client_id", "client_name", "address", "industry"]
df = pd.DataFrame(client_records, columns = h1)
display(df)


Query successful
MySQL Database connection successful


,client_id,client_name,address,industry
0,101,Big Business Federation,"23 Fingiertweg, 14534 Berlin",NGO
1,102,eCommerce GmbH,"27 Ersatz Allee, 10317 Berlin",Retail
2,103,AutoMaker AG,"20 Künstlichstraße, 10023 Berlin",Auto
3,104,Banko Bank,"12 Betrugstraße, 12345 Berlin",Banking
4,105,WeMoveIt GmbH,"138 Arglistweg, 10065 Berlin",Logistics


In [28]:
"""
Course 20, 'Fortgeschrittenes Russisch' (that's 'Advanced Russian' to you and me), is coming to an end, 
so we need to remove it from our database.
"""
delete = """
DELETE FROM course
WHERE course_id = 20;
"""
q1 = """
SELECT * 
FROM course;
"""
execute_query(connection, delete)
results = read_query(connection, q1)
course_records = []
for item in results:
    course_records.append(list(item))
h1 = ["course_id", "course_name", "language", "level", "duration", "start_date", "in_school", "teacher", "client"]
df = pd.DataFrame(course_records, columns =h1)
display(df)

Query successful


,course_id,course_name,language,level,duration,start_date,in_school,teacher,client
0,12,English for Logistics,ENG,A1,10,2020-02-01,1,1,105
1,13,Beginner English,ENG,A2,40,2019-11-12,0,6,101
2,14,Intermediate English,ENG,B2,40,2019-11-12,0,6,101
3,15,Advanced English,ENG,C1,40,2019-11-12,0,6,101
4,16,Mandarin für Autoindustrie,MAN,B1,15,2020-01-15,1,3,103
5,17,Français intermédiaire,FRA,B1,18,2020-04-03,0,2,101
6,18,Deutsch für Anfänger,DEU,A2,8,2020-02-14,1,4,102
7,19,Intermediate English,ENG,B2,10,2020-03-29,0,1,104


In [34]:
"""
Our school has two new teachers, Hank and Sue. Add them to the database
"""
new_teachers = []
teacher_info = []
#   teacher_id INT PRIMARY KEY,
#     first_name VARCHAR(40) NOT NULL,
#     last_name VARCHAR(40) NOT NULL,
#     language_1 VARCHAR(3) NOT NULL,
#     language_2 VARCHAR(3),
#     dob DATE,
#     tax_id INT UNIQUE,
#     phone_no VARCHAR(20)
# Get the input from the teachers
n = int(input("No of new teachers"))
for i in range(0,n):
    teacher_id = input("Please enter your id: ")
    teacher_info.append(teacher_id)
    first_name = input("First Name: ")
    teacher_info.append(first_name)
    last_name = input("Last Name: ")
    teacher_info.append(last_name)
    language_1 = input("Language 1: ")
    teacher_info.append(language_1)
    language_2 = input("Language 2: ")
    if language_2 == "None":
        teacher_info.append(None)
    else:
        teacher_info.append(language_2)
    dob = input("Date of Birth(yyyy-mm-dd): ")
    teacher_info.append(dob)
    tax_id = int(input("Tax ID: "))
    teacher_info.append(tax_id)
    phone_no = input('Phone Number: ')
    teacher_info.append(phone_no)
    new_teachers.append(tuple(teacher_info))
    teacher_info = []
print(new_teachers)


No of new teachers2
Please enter your id: 7
First Name: Hank
Last Name: Dodson
Language 1: ENG
Language 2: None
Date of Birth(yyyy-mm-dd): 1991-12-23
Tax ID: 11111
Phone Number: +491772345678
Please enter your id: 8
First Name: Sue
Last Name: Perkins
Language 1: MAN
Language 2: ENG
Date of Birth(yyyy-mm-dd): 1976-02-02
Tax ID: 22222
Phone Number: +491443456432
[('7', 'Hank', 'Dodson', 'ENG', None, '1991-12-23', 11111, '+491772345678'), ('8', 'Sue', 'Perkins', 'MAN', 'ENG', '1976-02-02', 22222, '+491443456432')]


In [36]:
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [38]:
sql = '''
    INSERT INTO teacher (teacher_id, first_name, last_name, language_1, language_2, dob, tax_id, phone_no) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    '''
connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql, new_teachers)

MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '7' for key 'teacher.PRIMARY''


In [39]:
q1 = """
SELECT * 
FROM teacher;
"""
results = read_query(connection, q1)
teacher_records = []
for item in results:
    teacher_records.append(list(item))
h1 = ["teacher_id", "first_name", "last_name", "language_1", "language_2", "dob", "tax_id", "phone_number"]
df = pd.DataFrame(teacher_records, columns =h1)
display(df)

,teacher_id,first_name,last_name,language_1,language_2,dob,tax_id,phone_number
0,1,James,Smith,ENG,None,1985-04-20,12345,+491774553676
1,2,Stefanie,Martin,FRA,None,1970-02-17,23456,+491234567890
2,3,Steve,Wang,MAN,ENG,1990-11-12,34567,+447840921333
3,4,Friederike,Müller-Rossi,DEU,ITA,1987-07-07,45678,+492345678901
4,5,Isobel,Ivanova,RUS,ENG,1963-05-30,56789,+491772635467
5,6,Niamh,Murphy,ENG,IRI,1995-09-08,67890,+491231231232
6,7,Hank,Dodson,ENG,None,1991-12-23,11111,+491772345678
7,8,Sue,Perkins,MAN,ENG,1976-02-02,22222,+491443456432
